In [1]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_and_optimize_and_sample_petri_model,
    load_and_calibrate_and_optimize_and_sample_petri_model
)
import numpy as np

In [2]:
DEMO_PATH = "../../notebook/integration_demo/"
ASKENET_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"

## load_and_sample_petri_model

In [3]:
num_samples = 3
timepoints = [0.5, 1.0, 2.0, 3.0, 4.0]

# Run sampling w/o an intervention
samples = load_and_sample_petri_model(ASKENET_PATH, num_samples, timepoints=timepoints)

# Save results
samples.to_csv(os.path.join(DEMO_PATH, "results_petri/sample_results.csv"), index=False)

In [4]:
interventions = [(1.1, "beta", 1.0), (2.1, "gamma", 0.1), (1.3, "beta", 2.0), (1.4, "gamma", 0.3)]

intervened_samples = load_and_sample_petri_model(
    ASKENET_PATH, num_samples, timepoints=timepoints, interventions=interventions
)


# Save results
intervened_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/sample_results_w_interventions.csv"),
    index=False,
)

## load_and_calibrate_and_sample_petri_model

In [5]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 62.71157956123352
iteration 25: loss = 60.2327595949173
iteration 50: loss = 59.43513986468315
iteration 75: loss = 57.84722709655762
iteration 100: loss = 57.21866774559021
iteration 125: loss = 54.55526041984558
iteration 150: loss = 56.586472272872925
iteration 175: loss = 56.48748993873596
iteration 200: loss = 57.76699471473694
iteration 225: loss = 56.420393228530884
iteration 250: loss = 56.50965619087219
iteration 275: loss = 57.15613293647766
iteration 300: loss = 57.15349459648132
iteration 325: loss = 56.28747487068176
iteration 350: loss = 56.70499229431152
iteration 375: loss = 56.99318718910217
iteration 400: loss = 56.75043559074402
iteration 425: loss = 56.87058615684509
iteration 450: loss = 56.919875383377075
iteration 475: loss = 56.7110652923584
iteration 500: loss = 56.31728911399841
iteration 525: loss = 57.057708978652954
iteration 550: loss = 56.911702394485474
iteration 575: loss = 56.30956697463989


## load_and_optimize_and_sample_petri_model

In [ ]:
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
OBJFUN = lambda x: np.abs(x)
INTERVENTION = [(0.1, "beta")]
QOI = ("scenario2dec_nday_average", "Infected_sol", 2)
ouu_samples, opt_policy = load_and_optimize_and_sample_petri_model(
    ASKENET_PATH,
    num_samples,
    timepoints=timepoints,
    interventions=INTERVENTION,
    qoi=QOI,
    risk_bound=10.,
    objfun=OBJFUN,
    initial_guess=0.02,
    bounds=[[0.],[3.]],
    verbose=True,
)

# Save results
ouu_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/optimize_sample_results.csv"), index=False
)
print("Optimal policy:", opt_policy["policy"])
print("Estimated risk at optimal policy", opt_policy["risk"])

Time taken: (2.19e-01 seconds per model evaluation).
Performing risk-based optimization under uncertainty (using alpha-superquantile)
Estimated wait time 1643.2 seconds...
Optimization completed in time 302.05 seconds.
Optimal policy:	0.0
Post-processing optimal policy...
Estimated risk at optimal policy [0.7047186017036438]
Optimal policy: 0.0
Estimated risk at optimal policy [0.7047186017036438]


## load_and_calibrate_and_optimize_and_sample_petri_model

In [ ]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
OBJFUN = lambda x: np.abs(x)
INTERVENTION = [(0.1, "beta")]
QOI = ("scenario2dec_nday_average", "Infected_sol", 2)
ouu_cal_samples, opt_cal_policy = load_and_calibrate_and_optimize_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    interventions=INTERVENTION,
    qoi=QOI,
    risk_bound=10.,
    objfun=OBJFUN,
    initial_guess=0.02,
    bounds=[[0.],[3.]],
    verbose=True,
)

# Save results
ouu_cal_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrate_optimize_sample_results.csv"), index=False
)
print("Optimal policy after calibration:", opt_policy["policy"])
print("Estimated risk at optimal policy after calibration", opt_policy["risk"])

iteration 0: loss = 37.5659122467041
iteration 25: loss = 35.41392183303833
iteration 50: loss = 33.67425513267517
iteration 75: loss = 32.595038652420044
iteration 100: loss = 33.10420513153076
iteration 125: loss = 32.42216110229492
iteration 150: loss = 33.292160987854004
iteration 175: loss = 31.957093477249146
iteration 200: loss = 32.40791869163513
iteration 225: loss = 32.1630494594574
iteration 250: loss = 32.64620661735535
iteration 275: loss = 32.447582960128784
iteration 300: loss = 32.42112612724304
iteration 325: loss = 32.70798373222351
iteration 350: loss = 32.1295063495636
iteration 375: loss = 32.78153920173645
iteration 400: loss = 32.15399241447449
iteration 425: loss = 32.18069672584534
iteration 450: loss = 32.34887766838074
iteration 475: loss = 32.39561104774475
iteration 500: loss = 32.51458191871643
iteration 525: loss = 32.285157680511475
iteration 550: loss = 32.21916651725769
iteration 575: loss = 32.54395031929016
iteration 600: loss = 32.487563133239746
it